In [1]:
# Ibis and Pandas
import ibis 
import pandas as pd
import duckdb
import time
import timeit

ibis.options.interactive = True # Automatically exucutes queries at the end of each cell 

In [2]:
# Sample dataset
data = {
    'id': [1, 2, 3, 4],
    'value': [100, 200, 300, 400],
    'category': ['A', 'B', 'A', 'B']
}
data

{'id': [1, 2, 3, 4],
 'value': [100, 200, 300, 400],
 'category': ['A', 'B', 'A', 'B']}

In [3]:
# Pandas DataFrame
pandas_df = pd.DataFrame(data)
pandas_df

,id,value,category
0,1,100,A
1,2,200,B
2,3,300,A
3,4,400,B


In [4]:
type(pandas_df)

pandas.core.frame.DataFrame

In [5]:
# Ibis
# Connect Ibis to DuckDB
connection = ibis.duckdb.connect(database=":memory:") # memory enables you to work with a temporary, fast, in-memory database

# Register the Pandas DataFrame as a table
connection.register(pandas_df, table_name="example_table")

# Create an Ibis table expression
ibis_table = connection.table("example_table")
ibis_table

# Notice Ibis has no notion of an index: If you want to use the index, you will need to turn it into a column.

┏━━━━━━━┳━━━━━━━┳━━━━━━━━━━┓
┃ id    ┃ value ┃ category ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━━━━┩
│ int64 │ int64 │ string   │
├───────┼───────┼──────────┤
│     1 │   100 │ A        │
│     2 │   200 │ B        │
│     3 │   300 │ A        │
│     4 │   400 │ B        │
└───────┴───────┴──────────┘

In [6]:
type(ibis_table)

ibis.expr.types.relations.Table

In [7]:
# The data types of columns in Pandas is accessed using dtypes and returns a Series Object
# In Ibis the schema method returns an ibis.Schema object
(pandas_df.dtypes, ibis_table.schema())

(id           int64
 value        int64
 category    object
 dtype: object,
 ibis.Schema {
   id        int64
   value     int64
   category  string
 })

In [8]:
# For selecting columns you can use the same syntax in Pandas and Ibis
(pandas_df[["id", "value"]],   ibis_table[["id", "value"]])

(   id  value
 0   1    100
 1   2    200
 2   3    300
 3   4    400,
 ┏━━━━━━━┳━━━━━━━┓
 ┃ id    ┃ value ┃
 ┡━━━━━━━╇━━━━━━━┩
 │ int64 │ int64 │
 ├───────┼───────┤
 │     1 │   100 │
 │     2 │   200 │
 │     3 │   300 │
 │     4 │   400 │
 └───────┴───────┘)

In [9]:
# Groub-by and aggregation in Pandas
print(pandas_df.groupby('category')['value'].sum())

category
A    400
B    600
Name: value, dtype: int64


In [10]:
# Perform the same Group-by and aggregation in Ibis
ibis_query = ibis_table.group_by('category').aggregate(sum_value=ibis_table['value'].sum())
# ibis_result = ibis_query.execute()
ibis_query

┏━━━━━━━━━━┳━━━━━━━━━━━┓
┃ category ┃ sum_value ┃
┡━━━━━━━━━━╇━━━━━━━━━━━┩
│ string   │ int64     │
├──────────┼───────────┤
│ A        │       400 │
│ B        │       600 │
└──────────┴───────────┘

In [11]:
url = "https://raw.githubusercontent.com/roualdes/data/refs/heads/master/bike.csv"
bike_data = pd.read_csv(url)
bike_data.head(5)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [12]:
bike_data.shape

(731, 16)

In [13]:
# Repeat original dataset 10,000 times resulting in 7.31 million rows
large_bike_data = pd.concat([bike_data] * 10000, ignore_index=True)
large_bike_data.shape

(7310000, 16)

In [1]:
# Pandas section
start_time = time.time()

pandas_result = large_bike_data.groupby('season')['cnt'].sum()

pandas_time = time.time() - start_time
print(f'Pandas operation completed in {pandas_time:.4f} seconds.')
print(pandas_result)

NameError: name 'time' is not defined

In [17]:
# Ibis section

# Register the DataFrame as a table in DuckDB
connection.register(large_bike_data, table_name="bike_table")
# Create an Ibis table expression referring to the bike_table in DuckDB
ibis_table = connection.table("bike_table")

start_time = time.time()

# Group by season and aggregate the total count of bikes in cnt
ibis_query = ibis_table.group_by('season').aggregate(total_cnt=ibis_table['cnt'].sum())
ibis_result = ibis_query.execute()

ibis_time = time.time() - start_time
print(f'Ibis operation completed in {ibis_time:.4f} seconds.')
print(ibis_result)

Ibis operation completed in 0.0344 seconds.
   season    total_cnt
0       1   4713480000
1       4   8416130000
2       3  10611290000
3       2   9185890000


In [21]:

# Writing a query in Python using Ibis

con = ibis.duckdb.connect(database=":memory:")

data = pd.DataFrame({
    "id": [1, 2, 3],
    "name": ["Alice", "Bob", "Charlie"],
    "age": [25, 30, 35],
    "salary": [50000, 60000, 70000]
})

# Convert the DataFrame to an Ibis table (memtable)
employee = ibis.memtable(data)

# Register the Ibis table to the connection
con.create_table("employee", employee)

# Query the table
query = con.table("employee").filter(lambda x: x.age > 30)[["name", "salary"]]

# Execute the query
result = query.execute()

# Print the result
print(result)

# ____GENERATED SQL QUERY____
# SELECT name, salary
# FROM employee
# WHERE age > 30


      name  salary
0  Charlie   70000
